"""<br>
    @Author: Deven Gupta<br>
    @Date: 13-10-2024<br>
    @Last Modified by: Deven Gupta<br>
    @Last Modified time: 13-10-2024<br>
    @Title: Task2- Using product Reviews for (Inferring and Expanding) using Google Gemini.<br>
<br>
"""<br>

## Inferring and Expanding

Installing library for gemini -- `! pip install google-generativeai ` 

In [15]:
import os
import google.generativeai as genai
import csv
import time

In [16]:
# Configure the API key
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

# Create the model
generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 64,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    generation_config=generation_config,
)

In [19]:
# Function to process reviews
def process_reviews(input_file, output_file):
    """
    Description:
        Processes customer reviews from an input text file, infers the sentiment, generates replies,
        and predicts categories for the reviews, then writes the results to a CSV file.
    Parameters:
        input_file (str): Path to the input text file containing customer reviews.
        output_file (str): Path to the output CSV file where results will be stored.
    Returns: None
    """
    with open(input_file, 'r', encoding='utf-8') as f:
        lines = f.read().strip().split('\n')

    reviews = []
    for line in lines:
        if line.startswith('Review:'):
            review_text = line.replace('Review: ', '').strip().strip('"')
            product = lines[lines.index(line) - 1].replace('Product: ', '').strip()
            reviews.append((product, review_text))

    with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['product','prediction', 'review', 'sentiment', 'generated_reply']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for product, review in reviews:
            # Infer sentiment
            sentiment_session = model.start_chat(history=[])
            sentiment_response = sentiment_session.send_message(f"Determine the sentiment of this review only in one word (positive, negative, and neutral): {review}")
            sentiment = sentiment_response.text.strip()

            # Generate a reply
            reply_session = model.start_chat(history=[])
            reply_response = reply_session.send_message(f"Generate a reply in 30 words to this review (if positive thank you, and otherwise apologize for the inconvenience): {review}")
            generated_reply = reply_response.text.strip()

            # Predict the category (here, the category is the product itself)
            prediction = product

            # Write the results to CSV
            writer.writerow({
                'product': product,
                'prediction': prediction,
                'review': review,
                'sentiment': sentiment,
                'generated_reply': generated_reply  
            })
            time.sleep(8)


In [20]:
def main():
    process_reviews('reviews.txt', 'reviews_output.csv')

if __name__ == "__main__":
    main()